In [ ]:
import requests

# %%

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  
search_term = 'expense'  

# %%
# Step 1: Fetch available data series
response = requests.get(f'{url}{key}').json()
if 'Structure' not in response:
    print("Error: Unexpected API response for Dataflow.")
    print(response)  # Debugging: Print the response if it fails
    exit()

series_list = response['Structure']['Dataflows']['Dataflow']

# %%
# Step 2: Search for series matching the search term
matching_series = []
for series in series_list:
    series_name = series['Name']['#text']
    if search_term.lower() in series_name.lower():
        series_id = series['KeyFamilyRef']['KeyFamilyID']
        matching_series.append((series_name, series_id))
        print(f"Found series - Name: {series_name}, ID: {series_id}")

if not matching_series:
    print(f"No series found containing '{search_term}'.")
    exit()


# %%

# Step 3: Retrieve dimensions for each matching series
dimension_map = {}

for series_name, series_id in matching_series:
    key = f'DataStructure/{series_id}'
    response = requests.get(f'{url}{key}').json()

    if 'Structure' not in response or 'KeyFamilies' not in response['Structure']:
        print(f"Error: Unexpected response for DataStructure/{series_id}")
        print(response)
        continue

    dimensions = response['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']
    print(f"\nDimensions for series '{series_name}':")

    dimension_map[series_id] = []  # Store dimensions
    for n, dimension in enumerate(dimensions):
        codelist_id = dimension.get('@codelist', 'UNKNOWN')
        concept_ref = dimension.get('@conceptRef', 'UNKNOWN')
        dimension_map[series_id].append(codelist_id)
        print(f"  Dimension {n+1}: {concept_ref} (Codelist ID: {codelist_id})")




# %%
# Step 4: Fetch and display codes for each dimension
for series_name, series_id in matching_series:
    print(f"\nCodelists for series '{series_name}':")
    for codelist_id in dimension_map[series_id]:
        if codelist_id == "UNKNOWN":
            print(f"  Skipping unknown codelist in series {series_id}")
            continue

        key = f"CodeList/{codelist_id}"
        response = requests.get(f'{url}{key}').json()

        if 'Structure' not in response or 'CodeLists' not in response['Structure']:
            print(f"  No codelist found for ID: {codelist_id}")
            continue

        codelist_data = response['Structure']['CodeLists']['CodeList']
        if isinstance(codelist_data, list):
            codes = codelist_data[0]['Code']
        else:
            codes = codelist_data['Code']

        print(f"\n  Codelist ID: {codelist_id}")
        for code in codes:
            code_value = code['@value']
            code_description = code['Description']['#text']
            print(f"    Code: {code_value}, Description: {code_description}")

